# Working with Files

In [0]:
%run /Workspace/Users/guslagares.data@gmail.com/[learn]Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets


In [0]:
%python
files = dbutils.fs.ls(f"{dataset_bookstore}/customers-json")
display(files)

In [0]:
%python
bookstore = 'dbfs:/Volumes/workspace/default/bookstore_dataset/'


In [0]:
SELECT * FROM json.`dbfs:/Volumes/workspace/default/bookstore_dataset/customers-json/export_001.json`

In [0]:
SELECT * FROM json.`dbfs:/Volumes/workspace/default/bookstore_dataset/customers-json/export_*.json`

In [0]:
SELECT * FROM text.`dbfs:/Volumes/workspace/default/bookstore_dataset/customers-json/export_001.json`

In [0]:
SELECT * FROM binaryFile.`dbfs:/Volumes/workspace/default/bookstore_dataset/customers-json/export_001.json`

In [0]:
SELECT * FROM csv.`dbfs:/Volumes/workspace/default/bookstore_dataset/books-csv`

In [0]:
USE CATALOG dae_learn_path;

CREATE OR REPLACE TABLE books AS 
SELECT * FROM 
read_files(
  "dbfs:/Volumes/workspace/default/bookstore_dataset/books-csv",
  header => "true",
  delimiter => ";",
  format => "csv"
)
--LOCATION "dbfs:/Volumes/workspace/default/bookstore_dataset/books-csv"

In [0]:
USE dae_learn_path.default;

CREATE TABLE customers AS 
SELECT * FROM json.`dbfs:/Volumes/workspace/default/bookstore_dataset/customers-json/`;

DESCRIBE EXTENDED customers

# Write Tables

In [0]:
%python
dbutils.widgets.text("dataset_bookstore", dataset_bookstore)

In [0]:
USE CATALOG dae_learn_path;

In [0]:
CREATE TABLE orders AS 
SELECT *
FROM parquet.`dbfs:/Volumes/workspace/default/bookstore_dataset/orders/`;

In [0]:
SELECT * 
FROM orders

In [0]:
CREATE OR REPLACE TABLE orders AS 
SELECT *
FROM parquet.`dbfs:/Volumes/workspace/default/bookstore_dataset/orders/`;

In [0]:
DESC HISTORY orders

In [0]:
INSERT OVERWRITE orders
SELECT *
FROM parquet.`dbfs:/Volumes/workspace/default/bookstore_dataset/orders/`;

In [0]:
DESC HISTORY orders

In [0]:
INSERT OVERWRITE orders
SELECT *, current_timestamp()
FROM parquet.`dbfs:/Volumes/workspace/default/bookstore_dataset/orders/`;

In [0]:
INSERT INTO orders
SELECT *
FROM parquet.`dbfs:/Volumes/workspace/default/bookstore_dataset/orders-new/`;

In [0]:
SELECT COUNT(*) FROM orders

In [0]:
CREATE OR REPLACE TEMP VIEW customers_updates AS 
SELECT * FROM json.`dbfs:/Volumes/workspace/default/bookstore_dataset/customers-json-new`;

MERGE INTO customers c
USING customers_updates u
ON c.customer_id = u.customer_id
WHEN MATCHED AND c.email IS NULL AND u.email IS NOT NULL THEN
  UPDATE SET email = u.email, updated = u.updated
WHEN NOT MATCHED THEN INSERT *

In [0]:
CREATE OR REPLACE TEMP VIEW books_updates
  (
    book_id STRING,
    title STRING,
    author STRING,
    category STRING,
    price DOUBLE
  )
USING CSV 
OPTIONS(
  path = "dbfs:/Volumes/workspace/default/bookstore_dataset/books-csv-new",
  header = "true",
  delimiter = ";"
);

SELECT * FROM books_updates

In [0]:
MERGE INTO books b 
USING books_updates u
ON b.book_id = u.book_id AND b.title = u.title
WHEN NOT MATCHED AND u.category = 'Computer Science' THEN
  INSERT * EXCEPT(_rescued_data)


# Advanced Transformations

In [0]:
SELECT * FROM customers

In [0]:
DESCRIBE customers

In [0]:
SELECT customer_id, profile:first_name, profile:address:country
FROM customers

In [0]:
SELECT profile
FROM customers
LIMIT 1

In [0]:
CREATE OR REPLACE TEMP VIEW parsed_customers AS
  SELECT customer_id, from_json(profile, schema_of_json('{"first_name":"Thomas","last_name":"Lane","gender":"Male","address":{"street":"06 Boulevard Victor Hugo","city":"Paris","country":"France"}}')) AS profile_struct
  FROM customers;
  
SELECT * FROM parsed_customers

In [0]:
DESCRIBE parsed_customers

In [0]:
SELECT customer_id, profile_struct.first_name, profile_struct.address.country
FROM parsed_customers

In [0]:
CREATE OR REPLACE TEMP VIEW customers_final AS 
  SELECT customer_id, profile_struct.*
  FROM parsed_customers;

SELECT * FROM customers_final;

In [0]:
select order_id, customer_id, books
from orders

In [0]:
select order_id, customer_id, explode(books) as book
from orders

In [0]:
select customer_id, 
  collect_set(order_id) as orders_set,
  collect_set(books.book_id) as books_set
from orders
group by customer_id

In [0]:
select customer_id,
  collect_set(books.book_id) AS before_flatten,
  array_distinct(flatten(collect_set(books.book_id))) AS after_flatten
from orders
group by customer_id

In [0]:
CREATE OR REPLACE VIEW orders_enriched AS 
SELECT *
FROM (
  SELECT *, explode(books) AS book
  FROM orders) o
INNER JOIN books b
ON o.book.book_id = b.book_id;

SELECT * FROM orders_enriched

In [0]:
CREATE OR REPLACE TEMP VIEW orders_updates AS 
SELECT * FROM read_files(
  "dbfs:/Volumes/workspace/default/bookstore_dataset/orders-new",
  format => "parquet"
);

SELECT * FROM orders
UNION 
SELECT * EXCEPT(_RESCUED_DATA) FROM orders_updates

In [0]:
SELECT * FROM orders
INTERSECT 
SELECT * EXCEPT(_RESCUED_DATA) FROM orders_updates

In [0]:
SELECT * FROM orders
MINUS 
SELECT * EXCEPT(_RESCUED_DATA) FROM orders_updates

In [0]:
CREATE OR REPLACE TABLE transactions AS 
  SELECT * 
  FROM (
    SELECT 
      customer_id,
      book.book_id AS book_id,
      book.quantity AS quantity
    FROM orders_enriched
  ) PIVOT (
    sum(quantity) FOR book_id in (
      'B01', 'B02', 'B03', 'B04', 'B05', 'B06',
      'B07', 'B08', 'B09', 'B10', 'B11', 'B12'
    )
  );

SELECT * FROM transactions;

# High Order Functions and SQL UDFs


In [0]:
SELECT * 
FROM orders

In [0]:
SELECT 
  order_id,
  books,
  FILTER(books, i -> i.quantity >= 2) AS multiple_copies
FROM orders

In [0]:
SELECT order_id, multiple_copies
FROM (
  SELECT 
    order_id,
    books,
    FILTER(books, i -> i.quantity >= 2) AS multiple_copies
  FROM orders)
WHERE size(multiple_copies) > 0;

In [0]:
SELECT 
  order_id,
  books,
  TRANSFORM (
    books,
    b -> CAST(b.subtotal * 0.8 AS INT)
  ) AS subtotal_after_discount
FROM orders;

In [0]:
CREATE OR REPLACE FUNCTION get_url(email STRING)
RETURNS STRING

RETURN concat("https://www.", split(email, "@")[1])

In [0]:
SELECT email, get_url(email) domain
FROM customers

In [0]:
DESCRIBE FUNCTION get_url

In [0]:
DESCRIBE FUNCTION EXTENDED get_url

In [0]:
CREATE FUNCTION site_type(email STRING)
RETURNS STRING
RETURN CASE 
        WHEN email LIKE "%.com" THEN "Commercial business"
        WHEN email LIKE "%.org" THEN "Non-profits organization"
        WHEN email LIKE "%.edu" THEN "Education institution"
        ELSE concat("Unknow extenstion for domain: ", split(email, "@")[1])
      END;

In [0]:
SELECT email, site_type(email) AS domain_category
FROM customers